In [29]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [125]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [31]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [34]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [97]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h","price_change_percentage_7d","price_change_percentage_14d","price_change_percentage_30d","price_change_percentage_60d","price_change_percentage_200d","price_change_percentage_1y"]])

In [98]:
# Create a DataFrame with the scaled data
market_scaled_df = pd.DataFrame(market_data_scaled, columns=["price_change_percentage_24h","price_change_percentage_7d","price_change_percentage_14d","price_change_percentage_30d","price_change_percentage_60d","price_change_percentage_200d","price_change_percentage_1y"])

# Copy the crypto names from the original DataFrame
market_scaled_df["coin_id"] = df_market_data.index

# Set the coin_id column as index
market_scaled_df = market_scaled_df.set_index("coin_id")

# Display the scaled DataFrame
market_scaled_df.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [38]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [100]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=2)
    k_model.fit(market_scaled_df)
    inertia.append(k_model.inertia_)

In [101]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data = pd.DataFrame(elbow_data)
# Display coordinates
df_elbow_data.head(5)

,k,inertia
0,1,287.000000
1,2,198.571818
2,3,145.635748
3,4,79.022435
4,5,66.217780


In [102]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow_data.hvplot.line (
    x = "k",
    y = "inertia",
    title = "Elbow Curve",
    xticks = k
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 

# The best value for k is 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [103]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=7)

In [104]:
# Fit the K-Means model using the scaled DataFrame
model.fit(market_scaled_df)

KMeans(n_clusters=4, random_state=7)

In [105]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
k_4 = model.predict(market_scaled_df)

# Print the resulting array of cluster values.
k_4

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 2, 2, 0, 0, 3, 0, 2, 3,
       0, 0, 0, 0, 2, 0, 3, 0, 2, 0, 1, 2, 2, 2, 3, 0, 2, 0, 0],
      dtype=int32)

In [106]:
# Create a copy of the scaled DataFrame
market_predictions_df = market_scaled_df.copy()

In [107]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
market_predictions_df["clusters"]= k_4

# Display the copy of the scaled DataFrame
market_predictions_df.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,3
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,3


In [108]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.

market_predictions_df.hvplot.scatter (
    x= "price_change_percentage_24h",
    y= "price_change_percentage_7d",
    by= "clusters",
    hover_cols=["coin_id"],
    title= "Bitcoin Price Change Percentage 7 days vs 24 hours"
    )

:NdOverlay   [clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [109]:
# Create a PCA model instance and set `n_components=3`.
pca=PCA(n_components=2)

In [110]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
market_pca = pca.fit_transform(market_scaled_df)

# View the scaled PCA data
market_pca

array([[-0.60066733,  0.84276006],
       [-0.45826071,  0.45846566],
       [-0.43306981, -0.16812638],
       [-0.47183495, -0.22266008],
       [-1.15779997,  2.04120919],
       [-0.51653377,  1.38837748],
       [-0.45071134,  0.51769912],
       [-0.34559977,  0.72943939],
       [-0.64946792,  0.43216514],
       [-0.75901394, -0.20119979],
       [-0.24819846, -1.37625159],
       [-0.43840762, -0.17533654],
       [-0.69342533, -0.47381462],
       [ 0.06049915,  2.90940385],
       [-0.39335243, -0.10819197],
       [-0.79617564, -0.49440875],
       [ 0.06407452, -1.26982514],
       [-0.48901506, -0.73271912],
       [-0.3062723 ,  0.70341515],
       [-0.51352775, -0.14280239],
       [-0.36212044, -0.98691441],
       [-0.60426463,  0.82739764],
       [-0.4132956 , -0.67411527],
       [-0.40748304, -0.21250655],
       [ 0.60897382,  0.56353212],
       [-0.45021114, -0.15101945],
       [-0.76466522, -0.51788554],
       [-0.55631468, -1.93820906],
       [-0.42514677,

In [111]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [112]:
# Create a new DataFrame with the PCA data.
market_pca_df = pd.DataFrame(
    market_pca, columns=["PC1","PC2"]
)
market_pca_df.head(5)
# Copy the crypto names from the original scaled DataFrame
market_pca_df["coin_id"] = market_scaled_df.index

# Set the coin_id column as index
market_pca_df = market_pca_df.set_index("coin_id")

# Display the scaled PCA DataFrame
market_pca_df.head(5)

,PC1,PC2
coin_id,,
bitcoin,-0.600667,0.842760
ethereum,-0.458261,0.458466
tether,-0.433070,-0.168126
ripple,-0.471835,-0.222660
bitcoin-cash,-1.157800,2.041209


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [113]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [114]:
# Create an empty list to store the inertia values
inertia =[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k: 
    k_model=KMeans(n_clusters=i, random_state=7)
    k_model.fit(market_pca_df)
    inertia.append(k_model.inertia_)

In [115]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

# Display coordinates
df_elbow_pca.head(5)

,k,inertia
0,1,206.351201
1,2,123.717780
2,3,52.498746
3,4,32.090328
4,5,22.808331


In [116]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow_pca.hvplot.line (
    x = "k",
    y = "inertia",
    title = "Elbow Curve",
    xticks = k
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 3


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** yes, 3 does differ from 4

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [117]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=3, random_state=7)

# Fit the K-Means model using the scaled DataFrame
model.fit(market_scaled_df)

KMeans(n_clusters=3, random_state=7)

In [118]:
# Fit the K-Means model using the PCA data
model.fit(market_pca_df)

KMeans(n_clusters=3, random_state=7)

In [119]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
k_3 = model.predict(market_pca_df)

# Print the resulting array of cluster values.
k_3

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0],
      dtype=int32)

In [120]:
# Create a copy of the scaled PCA DataFrame
market_pca_df_predictions = market_pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
market_pca_df_predictions["clusters"]= k_3

# Display the copy of the scaled PCA DataFrame
market_pca_df_predictions

,PC1,PC2,clusters
coin_id,,,
bitcoin,-0.600667,0.842760,0
ethereum,-0.458261,0.458466,0
tether,-0.433070,-0.168126,0
ripple,-0.471835,-0.222660,0
bitcoin-cash,-1.157800,2.041209,0
binancecoin,-0.516534,1.388377,0
chainlink,-0.450711,0.517699,0
cardano,-0.345600,0.729439,0
litecoin,-0.649468,0.432165,0


In [122]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
market_pca_df_predictions.hvplot.scatter (
    x="PC1",
    y="PC2",
    by="clusters",
    hover_cols= "coin_id",
    title = "Scatter Plot by Coin - PCA=2"
)


:NdOverlay   [clusters]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [30]:
# Composite plot to contrast the Elbow curves
# YOUR CODE HERE!

In [31]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 